In [1]:
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image, ImageDraw
from torchvision import transforms
import mmcv
import numpy as np
from IPython import display

import sys
sys.path.append("../")
from model import create_model
import utils

%load_ext autoreload
%autoreload 2

In [2]:
model_path = '../model.pth'
video_path = './video.mp4'
threshold = 0.1

model = create_model()
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

if torch.cuda.is_available():
    model.to('cuda')

In [3]:
def pred(image, model):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)['out'][0]
        return output


In [4]:
frames_tracked = []

video = mmcv.VideoReader(video_path)
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')

    output = pred(frame, model)

    output = (output > threshold).type(torch.IntTensor)
    output = output.cpu().numpy()[0]

    result = np.where(output > 0)
    coords = list(zip(result[0], result[1]))

    for cord in coords:
        frame.putpixel((cord[1], cord[0]), (255, 0, 0))

    frames_tracked.append(frame)

Tracking frame: 302

In [5]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()